In [1]:
import os
import shutil
import wandb
import torch

# Step 1: Clear Environment Variables
os.environ.pop('WANDB_API_KEY', None)

# Step 2: Clear Wandb Config Directory
wandb_config_dir = os.path.expanduser("~/.config/wandb")
if os.path.exists(wandb_config_dir):
    shutil.rmtree(wandb_config_dir)

import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get('CUDA_VISIBLE_DEVICES'))

# Try to force PyTorch to see the GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

!nvidia-smi

print("PyTorch version:", torch.__version__)
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda if torch.cuda.is_available() else "N/A")

# Print information about available GPUs
if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available")

# Check if CUDA is initialized
print("Is CUDA initialized:", torch.cuda.is_initialized())

# If CUDA is available but not initialized, try to initialize it
if torch.cuda.is_available() and not torch.cuda.is_initialized():
    try:
        torch.cuda.init()
        print("CUDA initialized successfully")
    except Exception as e:
        print(f"Failed to initialize CUDA: {e}")

# Set the default tensor type to cuda if available
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    print("Default tensor type set to CUDA")
else:
    print("Using CPU tensors")

CUDA_VISIBLE_DEVICES: None
zsh:1: command not found: nvidia-smi
PyTorch version: 2.2.1+cu121
Is CUDA available: False
CUDA version: N/A
No GPU available
Is CUDA initialized: False
Using CPU tensors


# Operational

In [ ]:
#Load the numpy arrays
bi_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/bi_np.pkl')
erc_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/erc_np.pkl')
pyrome_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/pyrome_np.pkl')
wui_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/wui_np.pkl')

#Load the latitude and longitude matrices
lats_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/lats_np.pkl', nodata_value=None)
longs_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/longs_np.pkl', nodata_value=None)

print("Shapes of the loaded numpy arrays:")
print(f"BI: {bi_np_loaded.shape}")
print(f"ERC: {erc_np_loaded.shape}")
print(f"Pyrome: {pyrome_np_loaded.shape}")
print(f"WUI: {wui_np_loaded.shape}")
print("Shapes of the latitude and longitude matrices:")
print(f"Lats: {lats_np_loaded.shape}")

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from dask import delayed, compute
import dask.dataframe as dd

# Function to load shapefile and convert CRS
def load_shapefile(shapefile_path):
    pyromes_gdf = gpd.read_file(shapefile_path)
    pyromes_gdf = pyromes_gdf.to_crs(epsg=4326)
    return pyromes_gdf

# Load the shapefile
shapefile_path = '/content/drive/My Drive/CloudFire/data/pyrome_shp/Pyromes_CONUS_20200206.shp'
pyromes_gdf = load_shapefile(shapefile_path)

# Define the shape and ignition date
ignition_date = '2023-08-01'
shape = bi_np_loaded.shape

# Function to process each pixel
@delayed
def process_pixel(i, j):
    if np.isnan(bi_np_loaded[i, j]):
        return None

    pyrome = pyrome_np_loaded[i, j]
    pyrome_geom = pyromes_gdf[pyromes_gdf['PYROME'] == pyrome].geometry
    if pyrome_geom.empty:
        return None

    random_point = pyrome_geom.sample(1).centroid.iloc[0]

    return {
        'Ignition date': ignition_date,
        'Ignition time': np.nan,
        'Containment date': np.nan,
        'Containment time': np.nan,
        'State': np.nan,
        'Name': np.nan,
        'MTBS ID': np.nan,
        'Longitude': random_point.x,
        'Latitude': random_point.y,
        'Acres': 0,
        'BI': bi_np_loaded[i, j],
        'ERC': erc_np_loaded[i, j],
        'WUI proximity': wui_np_loaded[i, j],
        'Pyrome': pyrome,
        'X pixel': i,
        'Y pixel': j
    }

# Create a list of delayed tasks
tasks = [process_pixel(i, j) for i in range(shape[0]) for j in range(shape[1])]

# Compute the tasks in parallel
results = compute(*tasks)

# Filter out None results
results = [res for res in results if res is not None]

# Convert to a DataFrame
op_df = pd.DataFrame(results)

# Display the first few rows of the DataFrame
print(op_df.head())

# Save the DataFrame to a CSV file (optional)
# op_df.to_csv('wildfire_data.csv', index=False)


In [ ]:
op_df.head()

In [ ]:
proc_op_df = preprocess_data(op_df, fit=False)
proc_op_df.head()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming model, aug_proc_df, and var_range are already defined and available

# Convert the augmented data to tensor
augmented_data_tensor = torch.tensor(proc_op_df.drop(columns=['Acres']).values, dtype=torch.float32).to(device)

# Generate a range of potential wildfire sizes (log-scaled)
fire_sizes_log = torch.linspace(0, 10, steps=40).unsqueeze(1).to(device)

# Threshold for large fires (log-scaled)
large_fire_threshold_log = np.log1p(5000)

# Run inference
model.eval()
pdf_values = []

with torch.no_grad():
    for i in range(augmented_data_tensor.size(0)):
        if i % 100 == 0:
            print(f"Processing row {i+1}/{augmented_data_tensor.size(0)}")
        context_point = augmented_data_tensor[i].unsqueeze(0).repeat(fire_sizes_log.size(0), 1)
        log_probs = model.log_prob(fire_sizes_log, context=context_point)
        pdf_values.append(torch.exp(log_probs.cpu()).numpy())

# Reshape the PDF values to match the grid
#pdf_values = np.array(pdf_values).reshape(len(var_range), len(var_range), -1)
pdf_values = np.array(pdf_values)

In [ ]:
pdf_values = np.array(pdf_values)

In [ ]:
# Check the shape of pdf_values and op_df to ensure they match in the number of rows
assert pdf_values.shape[0] == len(op_df), "The number of rows in pdf_values must match the number of rows in op_df"

# Create new column names for each threshold
threshold_columns = [f'thres_{i}' for i in range(pdf_values.shape[1])]

# Add the columns to op_df
for idx, col_name in enumerate(threshold_columns):
    op_df[col_name] = pdf_values[:, idx]

# Display the first few rows of the updated DataFrame


In [ ]:
print(op_df.columns)
op_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming op_df and pdf_values are already defined

# Define the shape of the image
image_shape = (36, 46)

# Create the figure and axes for 40 subplots
fig, axes = plt.subplots(7, 9, figsize=(20, 15), constrained_layout=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Define thresholds
thresholds = np.exp(np.linspace(0, 10, 40)) - 1  # Transform back from log scale

# Loop over each threshold and plot the corresponding heatmap
for i, ax in enumerate(axes)[:-1]:
    # Create an empty grid with np.nan
    grid = np.full(image_shape, np.nan)

    # Fill the grid with the pdf values
    for idx, row in op_df.iterrows():
        x = int(row['X pixel'])
        y = int(row['Y pixel'])
        grid[x, y] = row[f'thres_{i}']

    # Plot the heatmap
    sns.heatmap(grid, ax=ax, cmap="viridis", cbar=i == 0, vmin=0, vmax=1)

    # Set the title with the threshold value
    ax.set_title(f'Threshold: {thresholds[i]:.2f} Acres')

    # Remove x and y ticks for clarity
    ax.set_xticks([])
    ax.set_yticks([])

# Add a single color bar for the entire figure
fig.colorbar(axes[0].collections[0], ax=axes, orientation='horizontal', fraction=0.02, pad=0.04)

# Show the plot
plt.suptitle('Probability of Wildfires Greater Than Threshold Acres', fontsize=16)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Assuming op_df and pdf_values are already defined

# Define the shape of the image
image_shape = (36, 46)

# Define thresholds and start at index 18
thresholds = np.exp(np.linspace(0, 10, 40)) - 1  # Transform back from log scale
start_idx = 18
thresholds = thresholds[start_idx:]

# Define color palette
colors = sns.color_palette("husl", len(thresholds))

# Number of columns and rows
ncols = 4
nrows = int(np.ceil(len(thresholds) / ncols))

# Create the figure and axes for the subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(20, 5 * nrows), constrained_layout=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop over each threshold and plot the corresponding heatmap
for i, ax in enumerate(axes):
    if i >= len(thresholds):
        ax.axis('off')  # Hide unused subplots
        continue

    threshold = thresholds[i]
    # Create an empty grid with np.nan
    grid = np.full(image_shape, np.nan)

    # Fill the grid with the pdf values
    for idx, row in op_df.iterrows():
        x = int(row['X pixel'])
        y = int(row['Y pixel'])
        grid[x, y] = row[f'thres_{i + start_idx}']

    # Calculate min, max, and average
    valid_values = grid[~np.isnan(grid)]
    min_val = np.min(valid_values)
    max_val = np.max(valid_values)
    avg_val = np.mean(valid_values)

    # Plot the heatmap
    sns.heatmap(grid, ax=ax, cmap=sns.color_palette("coolwarm", as_cmap=True), cbar=i == 0, vmin=0, vmax=1)

    # Set the title with the threshold value, min, max, and average
    ax.set_title(f'Threshold: {threshold:.2f} Acres\nMin: {min_val:.2f}, Max: {max_val:.2f}, Avg: {avg_val:.2f}')

    # Remove x and y ticks for clarity
    ax.set_xticks([])
    ax.set_yticks([])

# Add a single color bar for the entire figure
fig.colorbar(axes[0].collections[0], ax=axes, orientation='horizontal', fraction=0.02, pad=0.04)

# Show the plot
plt.suptitle('Probability of Wildfires Greater Than Threshold Acres', fontsize=16)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Assuming op_df and pdf_values are already defined

# Define the shape of the image
image_shape = (36, 46)

# Calculate the 99th percentile fire size for each pixel
fire_sizes_99th = np.empty(image_shape)
fire_sizes_99th.fill(np.nan)

for idx, row in op_df.iterrows():
    x = int(row['X pixel'])
    y = int(row['Y pixel'])
    pdf_vals = row[[f'thres_{i}' for i in range(40)]].values
    cdf_vals = np.cumsum(pdf_vals)
    cdf_vals /= cdf_vals[-1]  # Normalize to [0, 1]
    fire_size_index = np.searchsorted(cdf_vals, 0.99)
    fire_sizes_99th[x, y] = np.exp(np.linspace(0, 10, 40)[fire_size_index]) - 1  # Transform back from log scale

# Plot the heatmap of the 99th percentile fire sizes
plt.figure(figsize=(16, 10))
sns.heatmap(fire_sizes_99th, cmap='viridis', cbar_kws={'label': '99th Percentile Fire Size (Acres)'}, vmin=0)
plt.title('99th Percentile Fire Size for Each Pixel')
plt.xlabel('X pixel')
plt.ylabel('Y pixel')
plt.show()

# Create a DataFrame for 99th percentile fire sizes per pyrome
pyrome_99th_fire_sizes = []

for pyrome in op_df['Pyrome'].unique():
    pyrome_data = op_df[op_df['Pyrome'] == pyrome]
    pyrome_fire_sizes = []
    for idx, row in pyrome_data.iterrows():
        x = int(row['X pixel'])
        y = int(row['Y pixel'])
        fire_size = fire_sizes_99th[x, y]
        if not np.isnan(fire_size):
            pyrome_fire_sizes.append(fire_size)
    if pyrome_fire_sizes:
        pyrome_99th_fire_sizes.append((pyrome, np.mean(pyrome_fire_sizes)))

pyrome_99th_fire_sizes_df = pd.DataFrame(pyrome_99th_fire_sizes, columns=['Pyrome', '99th Percentile Fire Size'])

# Sort by 99th percentile fire size for better visualization
pyrome_99th_fire_sizes_df = pyrome_99th_fire_sizes_df.sort_values(by='99th Percentile Fire Size', ascending=False)

# Plot the results
plt.figure(figsize=(16, 10))
sns.barplot(data=pyrome_99th_fire_sizes_df, x='Pyrome', y='99th Percentile Fire Size', palette='viridis')
plt.title('99th Percentile Fire Size per Pyrome')
plt.xlabel('Pyrome')
plt.ylabel('99th Percentile Fire Size (Acres)')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Assuming op_df and pdf_values are already defined

# Convert the DataFrame to numeric values for percentile calculations
op_df_numeric = op_df.apply(pd.to_numeric, errors='coerce')

# Calculate the 99th percentile for each pyrome
percentiles = {}
for pyrome in op_df['Pyrome'].unique():
    pyrome_data = op_df_numeric[op_df['Pyrome'] == pyrome]
    # Stack the threshold columns and compute the 99th percentile
    thres_values = pyrome_data.filter(like='thres_').values.flatten()
    percentiles[pyrome] = np.percentile(thres_values, 99)

# Create a DataFrame for plotting
percentiles_df = pd.DataFrame(list(percentiles.items()), columns=['Pyrome', '99th Percentile'])

# Sort by 99th percentile value for better visualization
percentiles_df = percentiles_df.sort_values(by='99th Percentile', ascending=False)

# Plot the results
plt.figure(figsize=(16, 10))
sns.barplot(data=percentiles_df, x='Pyrome', y='99th Percentile', palette='viridis')
plt.title('99th Percentile Max Fire Size per Pyrome')
plt.xlabel('Pyrome')
plt.ylabel('99th Percentile Max Fire Size (Probability)')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()


# GPD

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np
# Load the dataset
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df['Ignition date'] = pd.to_datetime(df['Ignition date'])
df['Containment date'] = pd.to_datetime(df['Containment date'])

# Calculate fire duration
df['Duration'] = (df['Containment date'] - df['Ignition date']).dt.days
df = df[['Latitude', 'Longitude', 'ERC', 'WUI proximity', 'BI', 'Pyrome', 'Acres', 'Ignition date']]

# Filter out the dataset to include only the Western United States
# Filter out if not in np.unique(pyrome_np_loaded)
df = df[df['Pyrome'].isin(np.unique(pyrome_np_loaded))]
#df = df[df['Pyrome'] < 60]
# Handle missing values (example: fill with mean)
df.fillna(df.mean(), inplace=True)
#Drop Rows with NaN values
df.dropna(inplace=True)

# Display the shape of the filtered dataset to confirm the filtering
print("Filtered dataset shape:", df.shape)



# Convert ERC, WUI, and BI to percentiles within each pyrome
def convert_to_percentile(df, column, group_by_column):
    df[f'{column}_percentile'] = df.groupby(group_by_column)[column].rank(pct=True) * 100
    return df

for col in ['ERC', 'WUI proximity', 'BI']:
    df = convert_to_percentile(df, col, 'Pyrome')

# One-hot encode the pyrome column
encoder = OneHotEncoder()
pyrome_encoded = encoder.fit_transform(df[['Pyrome']]).toarray()
pyrome_df = pd.DataFrame(pyrome_encoded, columns=encoder.get_feature_names_out(['Pyrome']))

# Compute the ignition date as cyclically encoded variables
def date_sin(date):
    return np.sin(2 * np.pi * date.timetuple().tm_yday / 366.)

def date_cos(date):
    return np.cos(2 * np.pi * date.timetuple().tm_yday / 366.)

df['ignition_sin'] = df['Ignition date'].apply(date_sin)
df['ignition_cos'] = df['Ignition date'].apply(date_cos)

# Combine all features into a single dataframe
model_df = pd.concat([df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres', 'ignition_sin', 'ignition_cos']], pyrome_df], axis=1)

# Standardize continuous variables
scaler = StandardScaler()
continuous_vars = model_df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos']]
scaled_vars = scaler.fit_transform(continuous_vars)
scaled_df = pd.DataFrame(scaled_vars, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos'])

# Combine scaled continuous variables and categorical variables
model_df = pd.concat([scaled_df, model_df[encoder.get_feature_names_out(['Pyrome'])], model_df['Acres']], axis=1)

# Filter out records with Acres <= 1000
model_df = model_df[model_df['Acres'] > 1000]

nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])


model_df.fillna(model_df.mean(), inplace=True)

print("Filtered model_df shape:", model_df.shape)

# Check for NaN values in model_df and print them if they exist
nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])

# Separate features (X) and target (y)
X = model_df.drop(columns=['Acres'])
y = model_df['Acres']


In [ ]:
model_df.head()

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K


# Check for NaN or infinite values in X and y
print("NaN in X:", np.any(np.isnan(X)))
print("Infinite in X:", np.any(np.isinf(X)))
print("NaN in y:", np.any(np.isnan(y)))
print("Infinite in y:", np.any(np.isinf(y)))

# Replace NaN or infinite values if any
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Define custom GPD loss function for XGBoost
import numpy as np
import xgboost as xgb
import scipy.stats as stats

# Define the GPD loss function for XGBoost
def gpd_loss(preds, dtrain):
    labels = dtrain.get_label()
    threshold = 2000  # example threshold
    excess = labels - threshold

    # Mask to identify values above the threshold
    mask = labels > threshold

    # Example shape parameter (could also be learned)
    xi = 0.1
    sigma = np.exp(preds)  # scale parameter from the model predictions

    # Apply mask to excess and sigma
    excess = excess[mask]
    sigma = sigma[mask]

    # Gradient calculation
    grad = -((1 + xi * (excess / sigma))**(-1/xi) * (excess / sigma + 1 / xi) / sigma)

    # Hessian calculation
    hess = grad * (grad / sigma + (1 + xi * (excess / sigma))**(-1/xi - 1) / sigma)

    # Initialize full gradient and hessian arrays with zeros
    full_grad = np.zeros_like(labels)
    full_hess = np.zeros_like(labels)

    # Set gradient and hessian for values above the threshold
    full_grad[mask] = grad
    full_hess[mask] = hess

    return full_grad, full_hess

# Load and preprocess your data here...

# Train XGBoost model with custom GPD loss function
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    'objective': 'reg:squarederror',  # using squared error as a baseline
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6
}

evals = [(dtrain, 'train'), (dtest, 'eval')]
evals_result = {}

bst = xgb.train(params, dtrain, num_boost_round=100, evals=evals, evals_result=evals_result, obj=gpd_loss, verbose_eval=True)

# Print training history
print(evals_result)

# Plot training and validation loss
import matplotlib.pyplot as plt

epochs = len(evals_result['train']['rmse'])
x_axis = range(0, epochs)

# Plot training and validation RMSE
plt.figure(figsize=(10, 6))
plt.plot(x_axis, evals_result['train']['rmse'], label='Train')
plt.plot(x_axis, evals_result['eval']['rmse'], label='Validation')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.title('XGBoost GPD Loss Training and Validation RMSE')
plt.legend()
plt.show()



In [ ]:
y_pred

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K


# Check for NaN or infinite values in X and y
print("NaN in X:", np.any(np.isnan(X)))
print("Infinite in X:", np.any(np.isinf(X)))
print("NaN in y:", np.any(np.isnan(y)))
print("Infinite in y:", np.any(np.isinf(y)))

# Replace NaN or infinite values if any
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the GPD loss function
def gpd_loss(y_true, y_pred):
    xi = 0.1  # Example shape parameter
    kappa = 0.95  # Example threshold parameter
    epsilon = 1e-6  # Small value to ensure numerical stability

    # Calculate the log scale parameter
    sigma = K.exp(y_pred) + epsilon  # Adding epsilon to avoid log(0)

    # Compute the GPD loss
    term1 = (xi + 1) / xi * K.log(1 + (y_true * ((1 - kappa)**(-xi) - 1)) / sigma + epsilon)
    term2 = K.log(xi * sigma / ((1 - kappa)**xi - 1) + epsilon)

    return K.mean(term1 + term2)

# Define the neural network architecture
def create_model(input_dim):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(input_dim,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))  # Output layer for the log scale parameter
    return model

# Instantiate the model
input_dim = X_train.shape[1]
model = create_model(input_dim)

# Compile the model with the custom GPD loss
model.compile(optimizer='adam', loss=gpd_loss)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Plot training and validation loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
!pip install elapid

In [ ]:
!pip install stemflow

In [ ]:
import elapid as ela
from stemflow.model.AdaSTEM import AdaSTEM, AdaSTEMRegressor
from stemflow.model_selection import ST_CV, ST_train_test_split
from stemflow.utils.plot_gif import make_sample_gif

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np
# Load the dataset
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df['Ignition date'] = pd.to_datetime(df['Ignition date'])
df['Containment date'] = pd.to_datetime(df['Containment date'])

# Calculate fire duration
df['Duration'] = (df['Containment date'] - df['Ignition date']).dt.days
df = df[['Latitude', 'Longitude', 'ERC', 'WUI proximity', 'BI', 'Pyrome', 'Acres', 'Ignition date']]

# Filter out the dataset to include only the Western United States
# Filter out if not in np.unique(pyrome_np_loaded)
df = df[df['Pyrome'].isin(np.unique(pyrome_np_loaded))]
#df = df[df['Pyrome'] < 60]
# Handle missing values (example: fill with mean)
df.fillna(df.mean(), inplace=True)
#Drop Rows with NaN values
df.dropna(inplace=True)

# Display the shape of the filtered dataset to confirm the filtering
print("Filtered dataset shape:", df.shape)



# Convert ERC, WUI, and BI to percentiles within each pyrome
def convert_to_percentile(df, column, group_by_column):
    df[f'{column}_percentile'] = df.groupby(group_by_column)[column].rank(pct=True) * 100
    return df

for col in ['ERC', 'WUI proximity', 'BI']:
    df = convert_to_percentile(df, col, 'Pyrome')

# One-hot encode the pyrome column
encoder = OneHotEncoder()
pyrome_encoded = encoder.fit_transform(df[['Pyrome']]).toarray()
pyrome_df = pd.DataFrame(pyrome_encoded, columns=encoder.get_feature_names_out(['Pyrome']))

# Compute the ignition date as cyclically encoded variables
def date_sin(date):
    return np.sin(2 * np.pi * date.timetuple().tm_yday / 366.)

def date_cos(date):
    return np.cos(2 * np.pi * date.timetuple().tm_yday / 366.)

df['ignition_sin'] = df['Ignition date'].apply(date_sin)
df['ignition_cos'] = df['Ignition date'].apply(date_cos)


# Combine all features into a single dataframe
model_df = pd.concat([df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres', 'ignition_sin', 'ignition_cos']], pyrome_df], axis=1)

# Standardize continuous variables
scaler = StandardScaler()
continuous_vars = model_df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos']]
scaled_vars = scaler.fit_transform(continuous_vars)
scaled_df = pd.DataFrame(scaled_vars, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos'])

# Combine scaled continuous variables and categorical variables
model_df = pd.concat([scaled_df, model_df[encoder.get_feature_names_out(['Pyrome'])], model_df['Acres']], axis=1)

# Filter out records with Acres <= 1000
model_df = model_df[model_df['Acres'] > 1000]

nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])


model_df.fillna(model_df.mean(), inplace=True)

print("Filtered model_df shape:", model_df.shape)

# Check for NaN values in model_df and print them if they exist
nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])

# Separate features (X) and target (y)
X = model_df.drop(columns=['Acres'])
y = model_df['Acres']
model_df.head()

# Extreme Value Plots

In [ ]:
# Fit and plot GPD for each Pyrome
pyrome_columns = [col for col in model_df.columns if 'Pyrome_' in col]

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(pyrome_columns) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))

for idx, pyrome_col in enumerate(pyrome_columns):
    # Filter data for the current Pyrome
    pyrome_data = model_df[model_df[pyrome_col] == 1]
    if pyrome_data.shape[0] > 0:
        y_pyrome = pyrome_data['Acres']

        # Fit GPD
        shape, loc, scale = stats.genpareto.fit(y_pyrome)

        # Generate values from the fitted GPD for the histogram range
        x = np.linspace(min(y_pyrome), max(y_pyrome), 1000)
        gpd_pdf = stats.genpareto.pdf(x, shape, loc, scale) * len(y_pyrome) * (max(y_pyrome) - min(y_pyrome)) / 50  # scale to match the histogram

        # Goodness of fit metric (Kolmogorov-Smirnov statistic)
        ks_statistic, ks_p_value = stats.kstest(y_pyrome, 'genpareto', args=(shape, loc, scale))

        # Plot histogram of wildfire sizes with GPD overlay
        ax = axes[idx // n_cols, idx % n_cols]
        ax.hist(y_pyrome, bins=50, edgecolor='k', alpha=0.7, label='Wildfire Sizes')
        ax.plot(x, gpd_pdf, 'r-', lw=2, label='Fitted GPD')
        ax.set_xlabel('Wildfire Size (Acres)')
        ax.set_ylabel('Frequency')
        #log scale
        ax.set_yscale('log')
        ax.set_title(f'{pyrome_col}\nSamples: {len(y_pyrome)}, Shape: {shape:.2f}, Loc: {loc:.2f}, Scale: {scale:.2f}\nKS Statistic: {ks_statistic:.2f}, p-value: {ks_p_value:.2f}')
        ax.legend()


# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Fit a Generalized Pareto Distribution to the data
shape, loc, scale = stats.genpareto.fit(y)

# Generate values from the fitted GPD for the histogram range
x = np.linspace(min(y), max(y), 1000)
gpd_pdf = stats.genpareto.pdf(x, shape, loc, scale) * len(y) * (max(y) - min(y)) / 50  # scale to match the histogram

# Plot histogram of wildfire sizes with GPD overlay
plt.figure(figsize=(10, 6))
plt.hist(y, bins=50, edgecolor='k', alpha=0.7, label='Wildfire Sizes')
plt.plot(x, gpd_pdf, 'r-', lw=2, label='Fitted GPD')
plt.xlabel('Wildfire Size (Acres)')
plt.ylabel('Frequency')
plt.title('Histogram of Wildfire Sizes with Fitted GPD')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

# Plot 1: Histogram of Wildfire Sizes
plt.figure(figsize=(10, 6))
plt.hist(y, bins=50, edgecolor='k', alpha=0.7)
plt.xlabel('Wildfire Size (Acres)')
plt.ylabel('Frequency')
plt.title('Histogram of Wildfire Sizes')
plt.show()

# Plot 2: Q-Q Plot
shape, loc, scale = stats.genpareto.fit(y)
quantiles = np.linspace(0, 1, len(y))
sorted_data = np.sort(y)
gpd_quantiles = stats.genpareto.ppf(quantiles, shape, loc, scale)

plt.figure(figsize=(10, 6))
plt.plot(gpd_quantiles, sorted_data, 'o')
plt.plot([min(gpd_quantiles), max(gpd_quantiles)], [min(gpd_quantiles), max(gpd_quantiles)], 'r--')
plt.xlabel('Theoretical Quantiles (GPD)')
plt.ylabel('Sample Quantiles')
plt.title('Q-Q Plot of Wildfire Sizes vs GPD')
plt.show()

def mean_excess_function(data, thresholds):
    excess_means = []
    for threshold in thresholds:
        excesses = data[data > threshold] - threshold
        excess_means.append(np.mean(excesses))
    return excess_means

thresholds = np.linspace(min(y), max(y), 100)
mean_excess = mean_excess_function(y, thresholds)

# Plot 3: Mean Excess Plot
plt.figure(figsize=(10, 6))
plt.plot(thresholds, mean_excess, 'b-')
plt.xlabel('Threshold')
plt.ylabel('Mean Excess')
plt.title('Mean Excess Plot')
plt.show()

def hill_estimator(data, k):
    sorted_data = np.sort(data)
    n = len(sorted_data)
    hill_estimates = []
    for i in range(1, k + 1):
        hill_estimates.append(np.mean(np.log(sorted_data[n - i:n]) - np.log(sorted_data[n - i - 1])))
    return hill_estimates

k_values = range(1, len(y) // 2)
hill_estimates = hill_estimator(y, max(k_values))

# Plot 4: Hill Plot
plt.figure(figsize=(10, 6))
plt.plot(k_values, hill_estimates, 'g-')
plt.xlabel('k')
plt.ylabel('Hill Estimate')
plt.title('Hill Plot')
plt.show()


# GMM

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np
# Load the dataset
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df['Ignition date'] = pd.to_datetime(df['Ignition date'])
df['Containment date'] = pd.to_datetime(df['Containment date'])

# Calculate fire duration
df['Duration'] = (df['Containment date'] - df['Ignition date']).dt.days
df = df[['Latitude', 'Longitude', 'ERC', 'WUI proximity', 'BI', 'Pyrome', 'Acres', 'Ignition date']]

# Filter out the dataset to include only the Western United States
# Filter out if not in np.unique(pyrome_np_loaded)
df = df[df['Pyrome'].isin(np.unique(pyrome_np_loaded))]
#df = df[df['Pyrome'] < 60]
# Handle missing values (example: fill with mean)
df.fillna(df.mean(), inplace=True)
#Drop Rows with NaN values
df.dropna(inplace=True)

# Display the shape of the filtered dataset to confirm the filtering
print("Filtered dataset shape:", df.shape)



# Convert ERC, WUI, and BI to percentiles within each pyrome
def convert_to_percentile(df, column, group_by_column):
    df[f'{column}_percentile'] = df.groupby(group_by_column)[column].rank(pct=True) * 100
    return df

for col in ['ERC', 'WUI proximity', 'BI']:
    df = convert_to_percentile(df, col, 'Pyrome')

# One-hot encode the pyrome column
encoder = OneHotEncoder()
pyrome_encoded = encoder.fit_transform(df[['Pyrome']]).toarray()
pyrome_df = pd.DataFrame(pyrome_encoded, columns=encoder.get_feature_names_out(['Pyrome']))

# Compute the ignition date as cyclically encoded variables
def date_sin(date):
    return np.sin(2 * np.pi * date.timetuple().tm_yday / 366.)

def date_cos(date):
    return np.cos(2 * np.pi * date.timetuple().tm_yday / 366.)

df['ignition_sin'] = df['Ignition date'].apply(date_sin)
df['ignition_cos'] = df['Ignition date'].apply(date_cos)

# Combine all features into a single dataframe
model_df = pd.concat([df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres', 'ignition_sin', 'ignition_cos']], pyrome_df], axis=1)

# Standardize continuous variables
scaler = StandardScaler()
continuous_vars = model_df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos']]
scaled_vars = scaler.fit_transform(continuous_vars)
scaled_df = pd.DataFrame(scaled_vars, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos'])

# Combine scaled continuous variables and categorical variables
model_df = pd.concat([scaled_df, model_df[encoder.get_feature_names_out(['Pyrome'])], model_df['Acres']], axis=1)

# Filter out records with Acres <= 1000
model_df = model_df[model_df['Acres'] > 100]

nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])


model_df.fillna(model_df.mean(), inplace=True)

print("Filtered model_df shape:", model_df.shape)

# Check for NaN values in model_df and print them if they exist
nan_values = model_df.isna().sum()
print("NaN values in model_df:\n", nan_values[nan_values > 0])



In [ ]:
model_df.shape

In [ ]:
model_df.head()

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np
# Function to select the optimal number of components using BIC
def select_optimal_gmm(X, max_components=10):
    bic_scores = []
    models = []

    for n_components in range(1, max_components + 1):
        model = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
        model.fit(X)
        bic = model.bic(X)
        bic_scores.append(bic)
        models.append(model)

    optimal_index = np.argmin(bic_scores)
    optimal_model = models[optimal_index]

    return optimal_model, bic_scores, optimal_index + 1

# Extract features and target variable
X = model_df.drop(columns=['Acres']).values
y = model_df['Acres'].values

# Select the optimal number of components
optimal_gmm, bic_scores, optimal_n_components = select_optimal_gmm(X)

print(f"Optimal number of components: {optimal_n_components}")



In [ ]:
import matplotlib.pyplot as plt

# Function to plot BIC scores
def plot_bic_scores(bic_scores):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(bic_scores) + 1), bic_scores, marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('BIC Score')
    plt.title('BIC Scores for Different Number of Components')
    plt.show()

# Plot the BIC scores
plot_bic_scores(bic_scores)

In [ ]:
# Fit the GMM with the optimal number of components
optimal_gmm.fit(X)

# Predict the component probabilities for each sample
probs = optimal_gmm.predict_proba(X)

# Predict the component means for each sample
predictions = optimal_gmm.predict(X)

# Append the probabilities and predictions to the dataframe for analysis
model_df['GMM_Probabilities'] = list(probs)
model_df['GMM_Predictions'] = predictions


In [ ]:
# Create a DataFrame for the new data
data = {
    'Latitude': lats_np_loaded.flatten(),
    'Longitude': longs_np_loaded.flatten(),
    'ERC': erc_np_loaded.flatten(),
    'WUI proximity': wui_np_loaded.flatten(),
    'BI': bi_np_loaded.flatten(),
    'Pyrome': pyrome_np_loaded.flatten(),
    'Acres': 0,
    'Ignition date': pd.to_datetime('2022-07-01')  # Random date for testing
}
df_new = pd.DataFrame(data)
df_new = df_new.dropna()


# Convert ERC, WUI proximity, and BI to percentiles based on the original data
for col in ['ERC', 'WUI proximity', 'BI']:
    df_new = convert_to_percentile_based_on_original(df_new, df, col, 'Pyrome')

# Compute the ignition date as cyclically encoded variables
df_new['ignition_sin'] = df_new['Ignition date'].apply(date_sin)
df_new['ignition_cos'] = df_new['Ignition date'].apply(date_cos)

In [ ]:
df_new.head()

In [ ]:
import numpy as np

def convert_to_percentile_based_on_original(df_new, df_original, column, group_by_column):
    percentiles = df_original.groupby(group_by_column)[column].rank(pct=True) * 100
    df_new[f'{column}_percentile'] = np.interp(df_new[column], np.sort(df_original[column]), percentiles)
    return df_new

# Load the numpy arrays
bi_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/bi_np.pkl')
erc_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/erc_np.pkl')
pyrome_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/pyrome_np.pkl')
wui_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/wui_np.pkl')
lats_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/lats_np.pkl', nodata_value=None)
longs_np_loaded = load_np_array('/content/drive/My Drive/CloudFire/data/vars_50k/longs_np.pkl', nodata_value=None)

# Create a DataFrame for the new data
data = {
    'Latitude': lats_np_loaded.flatten(),
    'Longitude': longs_np_loaded.flatten(),
    'ERC': erc_np_loaded.flatten(),
    'WUI proximity': wui_np_loaded.flatten(),
    'BI': bi_np_loaded.flatten(),
    'Pyrome': pyrome_np_loaded.flatten(),
    'Acres': 0,
    'Ignition date': pd.to_datetime('2022-07-01'),  # Random date for testing
    'Index': np.arange(lats_np_loaded.size),  # Track original indices
    'x': np.tile(np.arange(lats_np_loaded.shape[1]), lats_np_loaded.shape[0]),
    'y': np.repeat(np.arange(lats_np_loaded.shape[0]), lats_np_loaded.shape[1])
}
df_new = pd.DataFrame(data)
df_new = df_new.dropna()


# Convert ERC, WUI proximity, and BI to percentiles based on the original data
for col in ['ERC', 'WUI proximity', 'BI']:
    df_new = convert_to_percentile_based_on_original(df_new, df, col, 'Pyrome')

# Compute the ignition date as cyclically encoded variables
df_new['ignition_sin'] = df_new['Ignition date'].apply(date_sin)
df_new['ignition_cos'] = df_new['Ignition date'].apply(date_cos)

print(df_new.head())
# One-hot encode the pyrome column
pyrome_encoded_new = encoder.transform(df_new[['Pyrome']]).toarray()
pyrome_df_new = pd.DataFrame(pyrome_encoded_new, columns=encoder.get_feature_names_out(['Pyrome']))


# Combine all features into a single dataframe
# Combine all features into a single dataframe
df_new = pd.concat([df_new[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos', 'Index', 'x', 'y']], pyrome_df_new], axis=1)

df_new = df_new.dropna()

# Standardize continuous variables using the original scaler
continuous_vars_new = df_new[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos']]
scaled_vars_new = scaler.transform(continuous_vars_new)
scaled_df_new = pd.DataFrame(scaled_vars_new, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos'])

# Combine scaled continuous variables and categorical variables
df_new_scaled = pd.concat([scaled_df_new, df_new[encoder.get_feature_names_out(['Pyrome'])], df_new[['Index', 'x', 'y']]], axis=1)

print(df_new_scaled.shape)
df_new_scaled = df_new_scaled.dropna()
print(df_new_scaled.shape)
# Run predictions through the GMM
X_new = df_new_scaled.drop(columns=['Index', 'x', 'y']).values
probs = optimal_gmm.predict_proba(X_new)
predictions = optimal_gmm.predict(X_new)

# Append the probabilities and predictions to the dataframe for analysis
df_new_scaled['GMM_Probabilities'] = list(probs)
df_new_scaled['GMM_Predictions'] = predictions

print(df_new_scaled.shape)

In [ ]:
df_new_scaled.head(-40)


In [ ]:
# Ensure X_new is correctly formatted and scaled
continuous_vars_new = df_new[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos']]
scaled_vars_new = scaler.transform(continuous_vars_new)
scaled_df_new = pd.DataFrame(scaled_vars_new, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'ignition_sin', 'ignition_cos'])

# Combine scaled continuous variables and categorical variables
df_new_scaled = pd.concat([scaled_df_new, df_new[encoder.get_feature_names_out(['Pyrome'])], df_new[['Index', 'x', 'y']]], axis=1)

# Drop any remaining NaN values
df_new_scaled = df_new_scaled.dropna()

# Prepare X_new for GMM prediction
X_new = df_new_scaled.drop(columns=['Index', 'x', 'y']).values

# Run predictions through the GMM
probs = optimal_gmm.predict_proba(X_new)
predictions = optimal_gmm.predict(X_new)

# Initialize arrays to hold the results
probs_reshaped = np.full((36, 46, probs.shape[1]), np.nan)
predictions_reshaped = np.full(lats_np_loaded.shape, np.nan)
print(probs_reshaped.shape)
print(predictions_reshaped.shape)
print(df_new_scaled.shape)
# Reshape predictions to match the original 2D grid format
# Place the predictions back into the original 2D grid format
for i, row in df_new_scaled.iterrows():
    x = int(row['x'])
    y = int(row['y'])
    #print(i, x, y)
    # if i > 968:
    #   print(i, x, y)
    #   continue
    probs_reshaped[y, x, :] = probs[i]
    predictions_reshaped[y, x] = predictions[i]

# Display the shapes of the prediction arrays
print("Shapes of the prediction arrays:")
print(f"Probs: {probs_reshaped.shape}")
print(f"Predictions: {predictions_reshaped.shape}")

# Create a figure with two subplots for predictions
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot GMM Component Probabilities (First Component as Example)
axs[0].imshow(probs_reshaped[..., 0], cmap='viridis')
axs[0].set_title('GMM Component Probabilities (Component 1)')
axs[0].axis('off')

# Plot GMM Predictions
axs[1].imshow(predictions_reshaped, cmap='viridis')
axs[1].set_title('GMM Predictions')
axs[1].axis('off')

# Adjust layout
plt.tight_layout()
plt.show()

# Function to sample from the GMM to predict fire size
def predict_fire_size(gmm, X, num_samples=1):
    means = gmm.means_
    covariances = gmm.covariances_
    weights = gmm.weights_

    # Assuming the last dimension corresponds to fire size
    fire_size_samples = np.zeros((X.shape[0], num_samples))

    for i in range(X.shape[0]):
        component = np.random.choice(len(weights), p=gmm.predict_proba(X[i].reshape(1, -1))[0])
        mean = means[component, -1]
        covariance = covariances[component, -1, -1]

        fire_size_samples[i, :] = np.random.normal(mean, np.sqrt(covariance), num_samples)

    return fire_size_samples

# Predict fire size for the new data
fire_size_predictions = predict_fire_size(optimal_gmm, X_new, num_samples=1)

# Reshape fire size predictions to match the original 2D grid format
fire_size_predictions_reshaped = np.full(lats_np_loaded.shape, np.nan)

for i, row in df_new_scaled.iterrows():
    x = int(row['x'])
    y = int(row['y'])
    fire_size_predictions_reshaped[y, x] = fire_size_predictions[i]

# Plot the fire size predictions
plt.figure(figsize=(10, 6))
plt.imshow(fire_size_predictions_reshaped, cmap='viridis')
plt.title('Predicted Fire Sizes')
plt.colorbar(label='Fire Size (Acres)')
plt.axis('off')
plt.show()

In [ ]:
probs.shape

In [ ]:

# Reshape predictions to match the original data shape
probs_reshaped = probs.reshape(lats_np_loaded.shape + (-1,))
predictions_reshaped = predictions.reshape(lats_np_loaded.shape)

# Display the shapes of the prediction arrays
print("Shapes of the prediction arrays:")
print(f"Probs: {probs_reshaped.shape}")
print(f"Predictions: {predictions_reshaped.shape}")

# Create a figure with two subplots for predictions
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot GMM Component Probabilities (First Component as Example)
axs[0].imshow(probs_reshaped[..., 0], cmap='viridis')
axs[0].set_title('GMM Component Probabilities (Component 1)')
axs[0].axis('off')

# Plot GMM Predictions
axs[1].imshow(predictions_reshaped, cmap='viridis')
axs[1].set_title('GMM Predictions')
axs[1].axis('off')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
df_new_scaled.head(30)

## GMM PLots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.mixture import GaussianMixture

# Function to select the optimal number of components using BIC
def select_optimal_gmm(X, max_components=10):
    bic_scores = []
    models = []

    for n_components in range(1, max_components + 10):
        model = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
        model.fit(X)
        bic = model.bic(X)
        bic_scores.append(bic)
        models.append(model)

    optimal_index = np.argmin(bic_scores)
    optimal_model = models[optimal_index]

    return optimal_model, bic_scores, optimal_index + 1

# Extract the target variable (fire sizes)
y = model_df['Acres'].values.reshape(-1, 1)

# Select the optimal number of components
optimal_gmm, bic_scores, optimal_n_components = select_optimal_gmm(y)

print(f"Optimal number of components: {optimal_n_components}")

# Fit the GMM with the optimal number of components
optimal_gmm.fit(y)

# Function to plot the PDF of the GMM
def plot_gmm_pdf(gmm, y, n_bins=50):
    # Create a range of values for fire sizes
    x = np.linspace(y.min(), y.max(), 1000)

    # Calculate the PDF for each component
    pdf = np.zeros_like(x)
    for mean, covariance, weight in zip(gmm.means_.flatten(), gmm.covariances_.flatten(), gmm.weights_):
        pdf += weight * norm.pdf(x, mean, np.sqrt(covariance))

    # Plot the PDF
    plt.figure(figsize=(10, 6))
    plt.hist(y, bins=n_bins, density=True, alpha=0.6, color='g', label='Actual Fire Sizes')
    plt.plot(x, pdf, color='red', label='GMM PDF')
    plt.axvline(np.percentile(y, 75), color='blue', linestyle='--', label='75th Percentile')
    plt.xlabel('Fire Size (Acres)')
    plt.ylabel('Density')
    plt.title('Probability Density Function of Fire Sizes')
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to analyze the frequency of GMM predictions
def analyze_gmm_predictions(gmm, y):
    # Predict the component probabilities for each sample
    probs = gmm.predict_proba(y)

    # Predict the component means for each sample
    predictions = gmm.predict(y)

    # Append the probabilities and predictions to the dataframe for analysis
    model_df['GMM_Probabilities'] = list(probs)
    model_df['GMM_Predictions'] = predictions

    # Calculate the frequency of predictions in the upper quartile
    upper_quartile_threshold = np.percentile(y, 75)
    upper_quartile_predictions = model_df[model_df['Acres'] >= upper_quartile_threshold]['GMM_Predictions'].value_counts(normalize=True)

    return upper_quartile_predictions

# Plot the PDF of the GMM
plot_gmm_pdf(optimal_gmm, y)

# Analyze the frequency of GMM predictions in the upper quartile
upper_quartile_freq = analyze_gmm_predictions(optimal_gmm, y)

print("Frequency of GMM predictions in the upper quartile of fire sizes:")
print(upper_quartile_freq)


In [ ]:
import matplotlib.pyplot as plt

# Extract the component indices and their proportions
components = upper_quartile_freq.index
proportions = upper_quartile_freq.values

# Create a bar plot
plt.figure(figsize=(12, 6))
plt.bar(components, proportions, color='skyblue')
plt.xlabel('GMM Component')
plt.ylabel('Proportion in Upper Quartile')
plt.title('Proportion of GMM Components in the Upper Quartile of Fire Sizes')
plt.grid(True)
plt.show()


In [ ]:
# Evaluate the log-likelihood
log_likelihood = optimal_gmm.score(X)
print(f"Log-likelihood of the model: {log_likelihood}")

# Visualize the distribution of predicted fire sizes for different components
import matplotlib.pyplot as plt

# Plot BIC scores
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(bic_scores) + 1), bic_scores, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('BIC Score')
plt.title('BIC Scores for Different Number of Components')
plt.show()

# Visualize the GMM components and fire size distribution
plt.figure(figsize=(10, 6))
for i in range(optimal_n_components):
    component_data = model_df[model_df['GMM_Predictions'] == i]
    plt.hist(component_data['Acres'], bins=30, alpha=0.5, label=f'Component {i + 1}')
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Frequency')
plt.title('Fire Size Distribution by GMM Components')
plt.legend()
plt.show()


In [ ]:
# Frequency of Fire Sizes by GMM Components (Log-scaled)
plt.figure(figsize=(14, 8))
for i in range(optimal_n_components):
    component_data = model_df[model_df['GMM_Predictions'] == i]
    plt.hist(component_data['Acres'], bins=30, alpha=0.6, label=f'Component {i + 1}', log=True)
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Frequency (Log-scaled)')
plt.title('Log-scaled Frequency of Fire Sizes by GMM Components')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Function to select the optimal number of components using BIC
def select_optimal_gmm(X, max_components=10):
    bic_scores = []
    models = []

    for n_components in range(1, max_components + 1):
        model = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
        model.fit(X)
        bic = model.bic(X)
        bic_scores.append(bic)
        models.append(model)

    optimal_index = np.argmin(bic_scores)
    optimal_model = models[optimal_index]

    return optimal_model, bic_scores, optimal_index + 1

# Extract features and target variable
X = model_df.drop(columns=['Acres']).values
y = model_df['Acres'].values

# Select the optimal number of components
optimal_gmm, bic_scores, optimal_n_components = select_optimal_gmm(X)

print(f"Optimal number of components: {optimal_n_components}")

# Fit the GMM with the optimal number of components
optimal_gmm.fit(X)

# Predict the component probabilities for each sample
probs = optimal_gmm.predict_proba(X)

# Predict the component means for each sample
predictions = optimal_gmm.predict(X)

# Append the probabilities and predictions to the dataframe for analysis
model_df['GMM_Probabilities'] = list(probs)
model_df['GMM_Predictions'] = predictions

# Extract the means and covariances of the components
means = optimal_gmm.means_
covariances = optimal_gmm.covariances_

# Print the means and covariances to debug
print("Component Means:")
print(means)

print("Component Covariances:")
print(covariances)

# Generate the PDF for fire sizes
x_values = np.linspace(min(y), max(y), 1000)
pdf_values = np.zeros_like(x_values)

# Print range of x_values for debugging
print("x_values range:")
print(x_values[:10], "to", x_values[-10:])

for mean, cov in zip(means, covariances):
    component_pdf = norm.pdf(x_values, mean[-1], np.sqrt(cov[-1, -1]))
    pdf_values += component_pdf

    # Print intermediate pdf values for debugging
    print("Intermediate component PDF values:")
    print(component_pdf[:10])

# Normalize the PDF
pdf_sum = pdf_values.sum()
if pdf_sum > 0:
    pdf_values /= pdf_sum
else:
    print("Warning: Sum of PDF values is zero or negative, indicating an issue.")

# Print final PDF values for debugging
print("Final PDF values (first 10):")
print(pdf_values[:10])

# Calculate the upper quartile threshold
upper_quartile_threshold = np.percentile(y, 75)

# Print upper quartile threshold for debugging
print("Upper Quartile Threshold:")
print(upper_quartile_threshold)

# Plot the PDF for fire sizes
plt.figure(figsize=(10, 6))
plt.plot(x_values, pdf_values, label='PDF of Fire Sizes')
plt.axvline(x=upper_quartile_threshold, color='r', linestyle='--', label='75th Percentile (Upper Quartile)')
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Probability Density')
plt.title('Probability Density Function (PDF) of Fire Sizes')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np

# This function generates continuous heatmaps for each GMM component, showing how the component's weights
# (responsibilities) are distributed across BI and ERC percentiles. The process involves:
# 1. Generating a grid of points over the range of BI and ERC percentiles.
# 2. Extracting the weights (responsibilities) for each GMM component from the dataframe.
# 3. Fitting a Kernel Density Estimation (KDE) using BI and ERC percentiles, weighted by the component's responsibilities.
# 4. Evaluating the KDE on the grid points to create a smooth density estimate.
# 5. Plotting the density estimate as a heatmap for each component, ensuring all plots are square and using a consistent color scale.

# This visualization helps in understanding how the GMM partitions the data across BI and ERC percentiles, highlighting
# the regions where each component has higher weights.


def plot_gmm_component_heatmaps(df, n_components, x_col, y_col, weights_col, grid_size=100):
    # Generate grid for plotting
    x_min, x_max = df[x_col].min(), df[x_col].max()
    y_min, y_max = df[y_col].min(), df[y_col].max()
    x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, grid_size), np.linspace(y_min, y_max, grid_size))
    grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()]).T

    # Initialize list to store KDE values for scaling color bars
    kde_values = []

    for i in range(n_components):
        # Extract weights for the current component
        weights = df[weights_col].apply(lambda x: x[i]).values

        # Fit KDE for the current component
        kde = gaussian_kde(df[[x_col, y_col]].T, weights=weights)

        # Evaluate KDE on the grid
        z = kde(grid_points.T).reshape(x_grid.shape)
        kde_values.append(z)

    # Determine the global minimum and maximum KDE values for consistent color bars
    z_min = min(z.min() for z in kde_values)
    z_max = max(z.max() for z in kde_values)

    plt.figure(figsize=(22, 22))

    # Create a grid of subplots
    for i in range(n_components):
      # ValueError: Number of rows must be a positive integer, not 4.0
        n_rows = int(np.ceil(n_components / 2))

        plt.subplot(n_rows, 2, i + 1)

        # Plot the heatmap with consistent color scale
        z = kde_values[i]
        contour = plt.contourf(x_grid, y_grid, z, levels=20, cmap='viridis', vmin=z_min, vmax=z_max)
        plt.colorbar(contour, label='Density')
        plt.xlabel(x_col)
        plt.ylabel(y_col)
        plt.title(f'GMM Component {i + 1} Weighting')
        plt.gca().set_aspect('equal', adjustable='box')

    plt.tight_layout()
    plt.show()

# Example usage:
plot_gmm_component_heatmaps(model_df, optimal_n_components, 'BI_percentile', 'ERC_percentile', 'GMM_Probabilities')



In [ ]:
# Heatmap of the Correlation Matrix for Each GMM Component
for i in range(optimal_n_components):
    component_data = model_df[model_df['GMM_Predictions'] == i]
    correlation_matrix = component_data[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres']].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f'Correlation Matrix for GMM Component {i + 1}')
    plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Box Plot of Fire Sizes by GMM Components
plt.figure(figsize=(14, 8))
sns.boxplot(x='GMM_Predictions', y='Acres', data=model_df)
plt.xlabel('GMM Component')
plt.ylabel('Fire Size (Acres)')
plt.yscale('log')
plt.title('Box Plot of Fire Sizes by GMM Components (Log-scaled)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

# Fit the GMM (assuming optimal_gmm is already fitted with the best number of components)
# If not already done:
# optimal_gmm = GaussianMixture(n_components=optimal_n_components, covariance_type='full', random_state=42)
# optimal_gmm.fit(X)

# Generate a range of fire sizes to evaluate the PDF
fire_size_range = np.linspace(model_df['Acres'].min(), model_df['Acres'].max(), 1000)

# Evaluate the PDF for each fire size using the GMM
log_prob = optimal_gmm.score_samples(fire_size_range.reshape(-1, 1))
pdf = np.exp(log_prob)

# Plot the PDF
plt.figure(figsize=(10, 6))
plt.plot(fire_size_range, pdf, label='GMM PDF')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Probability Density')
plt.title('Probability Density Function of Fire Sizes under the GMM')
plt.legend()
plt.show()


In [ ]:
# Histogram of Fire Sizes for Large Fires (Log-scaled)
large_fires = model_df[model_df['Acres'] > 50000]

plt.figure(figsize=(10, 6))
plt.hist(large_fires['Acres'], bins=30, alpha=0.7, color='red', log=True)
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Frequency (Log-scaled)')
plt.title('Log-scaled Frequency of Large Fire Sizes (> 50,000 Acres)')
plt.show()


# Learning PDFs

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KernelDensity

# Load the dataset
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df['Ignition date'] = pd.to_datetime(df['Ignition date'])
df['Containment date'] = pd.to_datetime(df['Containment date'])

# Calculate fire duration
df['Duration'] = (df['Containment date'] - df['Ignition date']).dt.days
df = df[['Latitude', 'Longitude', 'ERC', 'WUI proximity', 'BI', 'Pyrome', 'Acres']]

# Filter out the dataset to include only the Western United States
df = df[df['Pyrome'] <60]

# Display the shape of the filtered dataset to confirm the filtering
print(df.shape)

# Handle missing values (example: fill with mean)
df.fillna(df.mean(), inplace=True)

# Convert ERC, WUI, and BI to percentiles within each pyrome
def convert_to_percentile(df, column, group_by_column):
    df[f'{column}_percentile'] = df.groupby(group_by_column)[column].rank(pct=True) * 100
    return df

for col in ['ERC', 'WUI proximity', 'BI']:
    df = convert_to_percentile(df, col, 'Pyrome')

#df = df[df['Pyrome'] == 34]
print(df.shape)
# One-hot encode the pyrome column
encoder = OneHotEncoder()
pyrome_encoded = encoder.fit_transform(df[['Pyrome']]).toarray()
pyrome_df = pd.DataFrame(pyrome_encoded, columns=encoder.get_feature_names_out(['Pyrome']))

# Combine all features into a single dataframe
model_df = pd.concat([df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres']], pyrome_df], axis=1)

# Standardize continuous variables
scaler = StandardScaler()
continuous_vars = model_df[['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile']]
scaled_vars = scaler.fit_transform(continuous_vars)
scaled_df = pd.DataFrame(scaled_vars, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile'])

# Combine scaled continuous variables and categorical variables
model_df = pd.concat([scaled_df, model_df[encoder.get_feature_names_out(['Pyrome'])], model_df['Acres']], axis=1)

# Filter for Pyrome_34 = 1
#model_df = model_df[model_df['Pyrome_34'] == 1]



In [ ]:
model_df.head()

In [ ]:
# Extract features and target
X = model_df.drop('Acres', axis=1).values
y = model_df['Acres'].values.reshape(-1, 1)

# Combine features and target
data = np.hstack((X, y))

# Fit a KDE model
kde = KernelDensity(kernel='gaussian', bandwidth=0.5)
kde.fit(data)

# Function to sample from the KDE
def sample_kde(kde, num_samples):
    samples = kde.sample(num_samples)
    return samples

# Generate samples from the KDE
num_samples = 100000
samples = sample_kde(kde, num_samples)

# Separate the sampled features and target
sampled_features = samples[:, :-1]
sampled_acres = samples[:, -1]

# Reverse the standardization for continuous variables
scaled_features = scaler.inverse_transform(sampled_features[:, :5])
sampled_lat_long_erc_wui_bi = pd.DataFrame(scaled_features, columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile'])

# Decode pyrome categories
sampled_pyrome = encoder.inverse_transform(sampled_features[:, 5:])

# Combine all sampled data into a dataframe
sampled_df = pd.DataFrame(np.hstack((sampled_lat_long_erc_wui_bi, sampled_pyrome, sampled_acres.reshape(-1, 1))), columns=['Latitude', 'Longitude', 'ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Pyrome', 'Acres'])

# Display the first few rows of the sampled dataframe
sampled_df.head()

In [ ]:
X.shape

In [ ]:
# Create pair plots for the original and sampled data
def plot_pairplots(original_df, sampled_df, variables, pyrome):
    original_subset = original_df[original_df['Pyrome'] == pyrome][variables]
    sampled_subset = sampled_df[sampled_df['Pyrome'] == pyrome][variables]

    # Add a 'Dataset' column to distinguish between original and sampled data
    original_subset['Dataset'] = 'Original'
    sampled_subset['Dataset'] = 'Sampled'

    combined_df = pd.concat([original_subset, sampled_subset])

    sns.pairplot(combined_df, hue='Dataset', corner=True, plot_kws={'alpha': 0.5})
    plt.suptitle(f'Pair Plot for Pyrome {pyrome}', y=1.02)
    plt.show()

# Plot pair plots for key variables and each pyrome
variables = ['ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres']
for pyrome in df['Pyrome'].unique():
    plot_pairplots(df, sampled_df, variables, pyrome)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot KDE comparison for each pyrome
def plot_kde_comparison(original_df, sampled_df, variable, pyrome_column='Pyrome'):
    pyromes = original_df[pyrome_column].unique()
    for pyrome in pyromes:
        plt.figure(figsize=(14, 7))

        # Plot original data KDE
        sns.kdeplot(data=original_df[original_df[pyrome_column] == pyrome], x=variable, label='Original Data', fill=True)

        # Plot sampled data KDE
        sns.kdeplot(data=sampled_df[sampled_df[pyrome_column] == pyrome], x=variable, label='Sampled Data', fill=True)

        plt.title(f'KDE Plot of {variable} for Pyrome {pyrome}')
        plt.legend()
        plt.show()

# Convert the sampled_df 'Pyrome' back to categorical for proper comparison
sampled_df['Pyrome'] = sampled_df['Pyrome'].astype(int).astype(str)

# Plot KDE comparison for ERC, WUI proximity, BI, and Acres
for var in ['ERC_percentile', 'WUI proximity_percentile', 'BI_percentile', 'Acres']:
    plot_kde_comparison(df, sampled_df, var)
